In [1]:
import pandas as pd

In [2]:
gcap_data = "/oak/stanford/groups/maggiori/GCAP/data/"
metadata = pd.read_csv(f"{gcap_data}/shared/WRDS_Transcripts/output/latest/metadata.csv" )
metadata.head()

,key,date,companyname_primary,companyid_primary,gvkey_primary,companynames,companyids,gvkeys,version_date
0,1925421,2006-02-15,"Genzyme Corporation, Inc.",346012,[12233],"['Genzyme Corporation, Inc.']",[346012],[[12233]],2024-08-05
1,1935525,2006-04-27,WithSecure Oyj,883456,[234356],['WithSecure Oyj'],[883456],[[234356]],2024-08-05
2,1935526,2006-08-01,WithSecure Oyj,883456,[234356],['WithSecure Oyj'],[883456],[[234356]],2024-08-05
3,1935527,2006-10-24,WithSecure Oyj,883456,[234356],['WithSecure Oyj'],[883456],[[234356]],2024-08-05
4,2433319,2006-05-18,"Nordstrom, Inc.",32215,[7922],"['Nordstrom, Inc.']",[32215],[[7922]],2024-08-05


In [3]:
metadata['version_date'].value_counts()

version_date
2024-08-05    326868
2024-11-19     27991
2025-09-04     10350
2025-04-01      6448
2025-05-23      4378
2025-01-29      1864
2025-04-23      1724
Name: count, dtype: int64

In [3]:
df = metadata[(metadata['date'] >= '2025-01-01') & (metadata['date'] <= '2025-06-30')].sample(100)
df.shape

(100, 9)

In [4]:
import os
def read_transcript(wrds_path: str, file_path: str) -> str:
    transcript_path = os.path.join(wrds_path, file_path)
    with open(transcript_path, "r", encoding="utf-8", errors="replace") as f:
        return f.read()

In [5]:
df['year'] = df.date.map(lambda x: x[:4])
df['file_path'] = df.apply(lambda x: x.year + "/" + str(x.key) + ".txt", axis=1)
wrds_path = f"{gcap_data}/shared/WRDS_Transcripts/output/latest/texts"
df['transcript'] = df.apply(lambda x: read_transcript(wrds_path, x.file_path), axis=1)
df.head()

,key,date,companyname_primary,companyid_primary,gvkey_primary,companynames,companyids,gvkeys,version_date,year,file_path,transcript
358449,1916829742,2025-01-23,"OSI Systems, Inc.",366230,[65607],"['OSI Systems, Inc.']",[366230],[[65607]],2025-04-01,2025,2025/1916829742.txt,"HEADLINE= OSI Systems, Inc., Q2 2025 Earnings ..."
362298,1931419578,2025-02-27,Howden Joinery Group Plc,31473,[104898],['Howden Joinery Group Plc'],[31473],[[104898]],2025-04-01,2025,2025/1931419578.txt,"HEADLINE= Howden Joinery Group Plc, 2024 Earni..."
362624,1931908219,2025-02-24,GenusPlus Group Ltd,645788030,[345340],['GenusPlus Group Ltd'],[645788030],[[345340]],2025-04-01,2025,2025/1931908219.txt,"HEADLINE= GenusPlus Group Ltd, H1 2025 Earning..."
358887,1917348510,2025-02-05,REA Group Limited,4493607,[248015],['REA Group Limited'],[4493607],[[248015]],2025-04-01,2025,2025/1917348510.txt,"HEADLINE= REA Group Limited, H1 2025 Earnings ..."
366487,1938675836,2025-05-07,"Bentley Systems, Incorporated",25514,"[37076, 148392]","['Bentley Systems, Incorporated']",[25514],"[[37076, 148392]]",2025-05-23,2025,2025/1938675836.txt,"HEADLINE= Bentley Systems, Incorporated, Q1 20..."


In [6]:
df.to_parquet(f"{gcap_data}shared/ai_geo1/temp/transcripts/transcripts_2025_workshop_sample.parquet", index=False)